# Keywords extractor

In [1]:
import pandas as pd

In [2]:
post_df = pd.read_csv("./data/lemmatize_labeled_rusentiment_text_mystem.csv")
post_df.head()

,text,sentiment
0,украина это часть русь искусственно отделять к...,negative
1,говорить относительно небольшой коррупция обра...,negative
2,свой поляк сделать русский понятно литовский п...,neutral
3,фильм зритель любить диковинный соседка пятый ...,neutral
4,государство платить посмотреть денежный поток ...,negative


In [6]:
post_df = post_df.dropna(subset=["text"]).sample(frac=1).reset_index(drop=True)

In [7]:
positive_corpus = post_df[post_df["sentiment"] == "positive"]["text"].tolist()[:10000]
positive_corpus = ". ".join(positive_corpus)

## Rake

In [18]:
from nlp_rake import Rake
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stops = list(set(stopwords.words("russian")))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dmitry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
rake = Rake(stopwords=stops, max_words=3)

In [20]:
rake.apply(positive_corpus)[:10]

[('носок расходиться шов', 9.0),
 ('издеваться хдддд старый', 9.0),
 ('оказываться германия президент', 9.0),
 ('опасный арза спокойный', 9.0),
 ('относиться вспомнить посмеяться', 9.0),
 ('взаимно весело романтичный', 9.0),
 ('бессоница начинаться каникулы', 9.0),
 ('женский логика предложение', 9.0),
 ('побег вроде писать', 9.0),
 ('бывший вспоминать улыбка', 9.0)]

## Yake

In [21]:
import yake

In [24]:
extractor = yake.KeywordExtractor(
    lan="ru",
    n=3,
    dedupLim=0.9,
    top=10,
)

In [25]:
extractor.extract_keywords(positive_corpus)

[('размер', 0.000102895758618097),
 ('размер подходить', 0.000189205165164366),
 ('хороший', 0.00019133980043983356),
 ('качество', 0.00019167285253150247),
 ('приходить', 0.0002124934927752914),
 ('качество хороший', 0.0002426607891507971),
 ('продавец', 0.0002480000815403851),
 ('заказывать размер', 0.00026863440417239587),
 ('размер заказывать размер', 0.00027965512773646384),
 ('приходить быстро', 0.0002865976350048858)]

## TextRank

In [11]:
from summa import keywords

In [17]:
clean_text_df = pd.read_csv("./data/lemmatize_labeled_rusentiment_text_mystem.csv")

In [18]:
clean_text_df = clean_text_df.dropna(subset=["text"]).sample(frac=1).reset_index(drop=True)

In [19]:
clean_positive_corpus = clean_text_df[clean_text_df["sentiment"] == "positive"]["text"].tolist()[:1000]

In [20]:
clean_positive_corpus = ". ".join(clean_positive_corpus)

In [22]:
keywords.keywords(clean_positive_corpus, language="russian").split("\n")[:10]

['эта',
 'болеть это',
 'свое',
 'приходить',
 'год выходить',
 'день',
 'спасибо',
 'сентябрь годить',
 'хороший',
 'хотеться']

## SpaCy

In [14]:
from tqdm import tqdm

import spacy

In [5]:
nlp = spacy.load("ru_core_news_lg")

In [9]:
max_chunk_size = nlp.max_length

In [12]:
document_chunks = [positive_corpus[i : i + max_chunk_size] for i in range(0, len(positive_corpus), max_chunk_size)]

In [15]:
docs = []
for document_chunk in tqdm(document_chunks):
    doc = nlp(document_chunk)
    docs.append(doc)

100%|██████████| 2/2 [01:15<00:00, 37.85s/it]


In [17]:
docs[0].ents[:10]

(дн спб,
 казахстан,
 республика казахстан,
 республика казахстан,
 республика казахстан,
 казахстан,
 республика казахстан,
 республика казахстан,
 альберт рау,
 казахстан)

In [26]:
from gensim.summarization import keywords

ModuleNotFoundError: No module named 'gensim.summarization'